# Packages 

In [61]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn.metrics import r2_score
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 
%matplotlib inline
from tqdm import tqdm
import csv as csv

In [96]:
output_path = '/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_immunotron/'
experiment_name = '20230306_MHV_HeLa_kinetics'
name_rawdata_file = '20230202_MHV_HeLa_sybr_plate'
plate_number='3'

# Import Data 

In [65]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}{}.txt'.format(output_path,experiment_name,name_rawdata_file,plate_number),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number))

#Import data
PCR_data_raw = pd.read_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})
                        
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)

#Define Titers and Timepoints
titers_list=[10,1,0.1,0.01,0.001,0.0001,0.00001,0]
timepoints_list=[24,20,16,12,8,5,4,3,2,1.5,1,0.5]

PCR_data

     Include  Color  Pos        Name     Cp  Concentration  Standard Status
0       True    255   A1    Sample 1  17.07            NaN         0    NaN
1       True    255   A2    Sample 2  17.40            NaN         0    NaN
2       True    255   A3    Sample 3  16.23            NaN         0    NaN
3       True    255   A4    Sample 4  16.80            NaN         0    NaN
4       True    255   A5    Sample 5  17.59            NaN         0    NaN
..       ...    ...  ...         ...    ...            ...       ...    ...
379     True  65280  P20  Sample 380    NaN            NaN         0    NaN
380     True  65280  P21  Sample 381    NaN            NaN         0    NaN
381     True  65280  P22  Sample 382    NaN            NaN         0    NaN
382     True    255  P23  Sample 383  25.13            NaN         0    NaN
383     True  65280  P24  Sample 384    NaN            NaN         0    NaN

[384 rows x 8 columns]


,Color,Tag,CT
0,255,A1,17.07
1,255,A2,17.40
2,255,A3,16.23
3,255,A4,16.80
4,255,A5,17.59
...,...,...,...
374,255,P15,32.13
375,255,P16,31.33
377,255,P18,30.77
378,255,P19,32.72


In [95]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}{}.txt'.format(output_path,experiment_name,name_rawdata_file,plate_number),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number))

#Import data
PCR_data_raw = pd.read_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)    


#define HK and GOI
PCR_data['GeneType']=['HK' if int(x[1:])<13 else 'GOI' for x in PCR_data['Tag']]
PCR_data = PCR_data.set_index(['GeneType'],append=True)

sub_df=PCR_data.query('GeneType=="HK"').reset_index().drop(['level_0'],axis=1)
sub_df['HK']=sub_df['CT']
sub_2df=PCR_data.query('GeneType=="GOI"').reset_index().drop(['level_0'],axis=1)
sub_df['GOI']=sub_2df['CT']
PCR_data=sub_df.drop(['CT','GeneType'],axis=1)

#set timepoints and titers
df_list=[]

PCR_data['numbers']=[int(x[1:]) for x in PCR_data['Tag']]
numbers_list=PCR_data['numbers'].unique()
for i,time in enumerate(timepoints_list):
    i=i+1
    sub_df = PCR_data.query('numbers==@i')
    sub_df['Timepoint']=time
    df_list.append(sub_df)
PCR_data=pd.concat(df_list)

df2_list=[]
PCR_data['letters']=[x[0] for x in PCR_data['Tag']]
letters_list=PCR_data['letters'].unique()
for i,titer in enumerate(titers_list):
    j=letters_list[0+2*i]
    k=letters_list[1+2*i]
    sub_df = PCR_data.query('letters==@j')
    sub_df['Titer']= titer
    df2_list.append(sub_df)
    sub_df = PCR_data.query('letters==@k')
    sub_df['Titer']= titer
    df2_list.append(sub_df)

PCR_data=pd.concat(df2_list)


#delta2CT method
df3_list=[]
PCR_data['DeltaCT']=PCR_data['GOI']-PCR_data['HK']
list_1=[PCR_data.query('letters=="O"')['DeltaCT']]
list_1.append([PCR_data.query('letters=="P"')['DeltaCT']])
delta=np.mean(list_2)
print(delta)


for number in numbers_list:
    sub_df=PCR_data.query('numbers==@number')
    sub_df['Delta2CT']=sub_df['DeltaCT']-delta
    df3_list.append(sub_df)
PCR_data=pd.concat(df3_list).drop(['numbers','letters'],axis=1)
PCR_data["GeneExpression"]=2**(-PCR_data['Delta2CT'])

#Mean of technical duplicate/triplicate 
list4_df=[]
for time in timepoints_list:
    sub_df=PCR_data.query('Timepoint==@time')
    for titer in titers_list:
        sub_sub_df = sub_df.query('Titer==@titer')
        mean_expression = np.mean(sub_sub_df['GeneExpression'])
        sub_sub_df['GeneExpression']=mean_expression
        mean_df=sub_sub_df.drop_duplicates('GeneExpression').drop(['Tag','HK','GOI','DeltaCT','Delta2CT','Color'],axis=1)
        list4_df.append(mean_df)
    PCR_data_mean = pd.concat(list4_df)

PCR_data_mean.sort_values(['Titer','Timepoint'],inplace=True)
PCR_data_mean.to_csv('{}/{}/Data/Analyzed_PCR_Data_Plate_{}.csv'.format(output_path,experiment_name,plate_number))

PCR_data_mean

11.368181818181819


,Timepoint,Titer,GeneExpression
122,1.0,0.00000,NaN
120,8.0,0.00000,NaN
119,24.0,0.00000,NaN
109,0.5,0.00001,NaN
108,1.0,0.00001,NaN
...,...,...,...
12,8.0,10.00000,89.771265
3,12.0,10.00000,670.076617
2,16.0,10.00000,14645.016549
1,20.0,10.00000,45645.653709
